## import package

In [7]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## 네이버 영화정보 url 수집

In [2]:
naver_df = pd.read_csv("total_movie.csv", index_col=0)

In [3]:
naver_df = naver_df[naver_df["movieNm"].str.contains("감독")==False]
naver_df = naver_df[naver_df["movieNm"].str.contains("확장")==False]
naver_df = naver_df.reset_index(drop=True)

In [31]:
def get_star_score_url(naver_df):
    url_df = pd.DataFrame(columns=["movieCd","movieNm_x","movieNm_y","url"])
    for n, i in enumerate(list(naver_df["movieNm"])):
        
        try:
            data = ("http://movie.naver.com/movie/search/result.nhn?query={movieNm}&section=all&ie=utf8").format(movieNm=i)
            r = requests.get(data)
            dom = BeautifulSoup(r.content, "html.parser")
            dom1 = dom.select("p.result_thumb")
            open_year = naver_df["prdtYear"].apply(lambda x: str(x)[0:4])
            if dom1[-1] == dom1[0]:
                for i, data in enumerate(dom1):
                    url = data.select_one("a")["href"]
                    movieNm = dom.select("dt")[i].text
                    url_df.loc[len(url_df)] = [
                    naver_df["movieCd"][n],
                    naver_df["movieNm"][n],
                    movieNm,
                    url,]
            elif dom1[-1] != dom1[0]:
                num = 0
                for j in range(0,len(dom.select("dd.etc")),2):
                    year = dom.select("dd.etc")[j].text[-4::]
                    if (year == open_year[n]) & (num == j):
                        num += 10
                        url = dom1[int(j/2)].select_one("a")["href"]
                        movieNm = dom.select("dt")[int(j/2)].text
                        url_df.loc[len(url_df)] = [
                        naver_df["movieCd"][n],
                        naver_df["movieNm"][n],
                        movieNm,
                        url,]
                    num += 2
        except:
            print(i)
    return url_df

In [34]:
url_df = get_star_score_url(naver_df)

로봇(애니)
트리플 X2 : 넥스트 레벨
유령신부(애니)
프라임 러브
링2(2005)
발리언트(애니)
펭귄 - 위대한 모험(다큐)
보글보글 스폰지밥(애니)
40살까지 못해본 사람
얼음왕국 : 북극의 여름이야기(다큐)
부그와 엘리엇(애니)
헨젤과 그레텔: 마녀 사냥꾼 3D
신나는 동물농장(애니)
4요일: 자살을 부르는 요일
국가대표 완결판-못 다한 이야기
아더와 미니모이-제1탄비밀원정대의출정
대한민국 1%
2008 서태지 심포니 위드 톨가 카쉬프 로열필하모닉
언터처블: 1%의 우정


### url_df to csv

In [40]:
url_df.to_csv("url.csv",encoding="utf-8")

## 네이버 영화 별점 수집

In [3]:
url_df = pd.read_csv("url.csv",index_col=0)

In [5]:
url_df.head()

,movieCd,movieNm_x,movieNm_y,url
0,20030371.0,태극기 휘날리며,태극기 휘날리며 (TaeGukGi: Brotherhood Of War),/movie/bi/mi/basic.nhn?code=36666
1,20040555.0,트로이,트로이 (Troy),/movie/bi/mi/basic.nhn?code=37742
2,20040673.0,내 머리 속의 지우개,내 머리 속의 지우개 (A Moment To Remember),/movie/bi/mi/basic.nhn?code=38662
3,20040649.0,귀신이 산다,귀신이 산다 (Ghost House),/movie/bi/mi/basic.nhn?code=38254
4,20159748.0,투모로우랜드,투모로우랜드 (Tomorrowland),/movie/bi/mi/basic.nhn?code=101960


### 네이버 영화 code 추출

In [8]:
url_df["code"] = url_df["url"].apply(lambda x:np.int(x.split("=")[1]))

In [11]:
def get_star_score(url_df):
    star_score_df = pd.DataFrame(columns=["movieCd","movieNm","star_score","star_user_count"])#,"scene_score","scene_count"])
    for n, i in enumerate(list(url_df["code"])):
        try:
            data = ("http://movie.naver.com/movie/bi/mi/point.nhn?code={code}").format(code=i)
            r = requests.get(data)
            dom = BeautifulSoup(r.content, "html.parser")
            dom1 = dom.select_one("#beforePointArea")
            dom2 = dom1.select("em")[2:]
            star_user_count = dom2[-1].text
            star_score = ("").join([i.text for i in dom2 [:-1]])
            star_score_df.loc[len(star_score_df)] = [
                url_df["movieCd"][n],
                url_df["movieNm_x"][n],
                star_score,
                star_user_count,
            ]
        except:
            print(i)
    return star_score_df

In [12]:
star_score_df = get_star_score(url_df)

60779
98446
98447


In [16]:
star_score_df["star_score"] = star_score_df["star_score"].astype(float)
star_score_df["star_user_count"] = star_score_df["star_user_count"].apply(lambda x: np.int(x.replace(",","")))

### star_score_df to csv

In [23]:
star_score_df.to_csv("star_score.csv",encoding="utf-8")